# Schedulify 🗓️
A multi-agent, stateful scheduling assistant built using Google ADK and Gemini.

In [1]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    print("✅ Gemini API key loaded from environment.")
else:
    print("❌ GOOGLE_API_KEY not found. Set it as an environment variable.")


In [2]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.tools import AgentTool, FunctionTool
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.adk.sessions import InMemorySessionService

print("✅ ADK components imported successfully.")

In [3]:
import logging
logging.getLogger("google_genai").setLevel(logging.ERROR)

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)


## Agent Tools (Inbox, Calendar, Reminder, Logs)

In [4]:
def read_inbox():
    return [
        {"sender": "Manager", "message": "Send project update by tomorrow."},
        {"sender": "HR", "message": "Your meeting is on Friday."},
        {"sender": "Teammate", "message": "Can you review my PR?"},
        {"sender": "Product Owner", "message": "Prepare a demo for next sprint planning."},
        {"sender": "DevOps", "message": "Server migration scheduled tonight."},
        {"sender": "Finance", "message": "Submit investment declaration before month end."}
    ]

inbox_tool = FunctionTool(func=read_inbox)


In [5]:
class DummyCalendar:
    def __init__(self):
        self.events = []

    def create_event(self, title: str, date: str, time: str, participants: list[str]):
        event = {
            "title": title,
            "date": date,
            "time": time,
            "participants": participants,
        }
        self.events.append(event)
        return {"status": "created", "event": event}

    def list_events(self):
        return {"events": self.events}

dummy_calendar = DummyCalendar()
calendar_create_tool = FunctionTool(dummy_calendar.create_event)
calendar_list_tool = FunctionTool(dummy_calendar.list_events)


In [6]:
class DummyReminderStore:
    def __init__(self):
        self.reminders = []

    def save_reminder(self, **kwargs):
        self.reminders.append(kwargs)
        return {"saved": True, "reminder": kwargs}

dummy_reminder = DummyReminderStore()
reminder_tool = FunctionTool(func=dummy_reminder.save_reminder)


In [7]:
class DummyLogStore:
    def __init__(self):
        self.logs = []

    def write_log(self, **kwargs):
        self.logs.append(kwargs)
        return {"logged": True, "data": kwargs}

log_tool = FunctionTool(func=DummyLogStore().write_log)


## Agents

In [8]:
inbox_agent = Agent(
    name="InboxAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
Summarize inbox messages and extract sender, topic, urgency.
""",
    tools=[inbox_tool],
    output_key="inbox_summary"
)


In [9]:
task_agent = Agent(
    name="TaskExtractorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Extract actionable tasks from inbox summaries.",
    tools=[],
    output_key="extracted_tasks"
)


In [10]:
reminder_agent = Agent(
    name="ReminderAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Create and store reminders from tasks.",
    tools=[reminder_tool],
    output_key="generated_reminders"
)


In [11]:
meeting_agent = Agent(
    name="MeetingSchedulerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Schedule meetings using calendar tools.",
    tools=[calendar_create_tool, calendar_list_tool],
    output_key="scheduled_meeting"
)


In [12]:
supervisor_agent = Agent(
    name="SupervisorAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
Orchestrate agents, maintain session memory, and produce final responses.
""",
    tools=[
        AgentTool(inbox_agent),
        AgentTool(task_agent),
        AgentTool(reminder_agent),
        AgentTool(meeting_agent),
        log_tool
    ],
    output_key="supervisor_plan"
)


## Stateful Runner

In [13]:
APP_NAME = "Schedulify"
USER_ID = "default"

session_service = InMemorySessionService()
runner = Runner(
    agent=supervisor_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("✅ Stateful agent initialized.")
